# GRIDR Conventions explained

In [ ]:
import numpy as np
from rasterio.features import rasterize, geometry_mask
from rasterio.transform import Affine
import shapely
print(shapely.__version__)
from shapely.geometry import Polygon
import rasterio
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook # enables plot interface in J notebook
# init bokeh
output_notebook()
from typing import Tuple, List, Union
from bokeh.models import ColumnDataSource, Label, Arrow, NormalHead, OpenHead, VeeHead

import sys
sys.path.insert(0,"../python")
from gridr.core.grid import grid_rasterize
from gridr.core.grid import grid_commons
from gridr.core.grid import grid_utils
display(grid_rasterize)

## Grid convention

Pixel centers are considered to be at whole coordinates.

In [ ]:
shape = (8, 12)
resolution = (3, 1)
origin = (0., 0.)

# Compute grids affected coordinates
cxx, cyy = grid_commons.grid_regular_coords_2d(shape, origin, resolution, sparse=False)
cx, cy = cxx[0,:], cyy[:,0]
cx, cy

In [ ]:
# Corresponding a pixels area map to illustrate
pixels = np.asarray([[shapely.geometry.Polygon( [(i-resolution[1]/2, j-resolution[0]/2),
                                      (i-resolution[1]/2, j-resolution[0]/2+resolution[0]),
                                      (i-resolution[1]/2+resolution[1], j-resolution[0]/2+resolution[0]),
                                      (i-resolution[1]/2+resolution[1], j-resolution[0]/2),
                                      (i-resolution[1]/2, j-resolution[0]/2),])
           for i in cx] for j in cy])

plot_res = 60
p = figure(width=shape[1]*plot_res, height=shape[0]*plot_res)
p.x_range.range_padding = p.y_range.range_padding = 0.5
p.y_range.flipped = True
p.cross(cxx.flatten(), cyy.flatten(), size=5, color="black", alpha=0.5)
p.cross(cxx[0,0], cyy[0,0], size=6, color="red", alpha=1)
geom_x, geom_y = [], []
[(geom_x.append(list(polygon.exterior.coords.xy[0])), geom_y.append(list(polygon.exterior.coords.xy[1]))) for polygon in pixels.flatten() ]
p.patches('x', 'y', source = ColumnDataSource(dict(x = geom_x, y = geom_y)), line_color = "grey", line_width = 0.2, fill_color=None)

origin_annotation = Label(x=0, y=0, x_units='data', y_units='data',
                 text='(0, 0)', x_offset=0, y_offset=-resolution[0]/4, text_align="center", text_alpha=0.7, text_color='black', text_baseline='bottom', text_font_size='12px' )
p.add_layout(origin_annotation)
vh = VeeHead(size=10, line_color="red", fill_color="red")
p.add_layout(Arrow(end=vh, x_start=0.0, y_start=0., x_end=1, y_end=0, line_color="red"))
p.add_layout(Arrow(end=vh, x_start=0.0, y_start=0., x_end=0, y_end=1, line_color="red"))

p.grid.grid_line_width = 0.5
show(p)

In [ ]:
# Lets add a vector geometry defined by the Polygon and its origin mapping convention towards the grid
geometry_origin = (0.5, 0.5)
epsilon = 0.001
geometry = shapely.geometry.Polygon([(3.5-epsilon,2.5-epsilon), (7.5+epsilon,2.5-epsilon), (7.5+epsilon,6.5+epsilon), (3.5-epsilon,6.5+epsilon)])
#geometry = shapely.geometry.Polygon([(-geometry_origin[1],(-geometry_origin[0])),
#                                      (shape[1]+geometry_origin[1],(-geometry_origin[0])),
#                                       (shape[1]+geometry_origin[1],(shape[0]+geometry_origin[0])),
#                                        (-geometry_origin[1],(shape[0]+geometry_origin[0]))])

#geometry_origin = (0.5, 0.5)
#geometry = geometry = shapely.geometry.Polygon([(2.,2.), (3.,2.), (3,4), (2,4)])
#geometry = geometry = shapely.geometry.Polygon([(2.,2.), (3.,2.), (3,3), (2,3)])
print(geometry)
# We want to hereby tell that the grid coordinates (0,0) corresponds to the geometry point (0.5, 0.5)

# Prepare plot convention
delta = np.array(geometry_origin) - np.array(origin)
geom_x, geom_y = [], []
[(geom_x.append(list((polygon.exterior.coords.xy[0]-delta[1]))), geom_y.append(list((polygon.exterior.coords.xy[1]-delta[0])))) for polygon in (geometry,) ]
try:
    # In order to manage update of polygon definition if the cell is executed more than once
    polygon_geometry.visible = False
except:
    pass
try:
    # In order to manage update of polygon definition if the cell is executed more than once
    raster_patch.visible = False
except:
    pass
polygon_geometry = p.patches('x', 'y', source = ColumnDataSource(dict(x = geom_x, y = geom_y)), line_color = "green", line_width = 3, fill_color=None, name="polygon_geometry")#fill_color="green", fill_alpha=0.5)
show(p)

In [ ]:
# Now call the rasterize method
alg = grid_rasterize.GridRasterizeAlg.RASTERIO_RASTERIZE
#alg = grid_rasterize.GridRasterizeAlg.SHAPELY
kwargs_alg = {}
if alg == grid_rasterize.GridRasterizeAlg.SHAPELY:
    kwargs_alg['shapely_predicate'] = grid_rasterize.ShapelyPredicate.COVERS

win=None
#win=np.array([(2,3),(5,6)])

raster = grid_rasterize.grid_rasterize(
        grid_coords=None,
        shape=shape,
        origin=geometry_origin,
        resolution=resolution,
        win=win,
        geometry=geometry,
        alg=alg,
        dtype=np.uint8,
        **kwargs_alg,
        )

# light up rasterize pixels
geom_x, geom_y = [], []
if win is None:
    for j,i in (zip(*np.where(raster==0))):
        geom_x.append(list(pixels[j,i].exterior.coords.xy[0]))
        geom_y.append(list(pixels[j,i].exterior.coords.xy[1]))
else:
    for j,i in (zip(*np.where(raster==0))):
        geom_x.append(list(pixels[j+win[0,0],i+win[1,0]].exterior.coords.xy[0]))
        geom_y.append(list(pixels[j+win[0,0],i+win[1,0]].exterior.coords.xy[1]))
    
try:
    # In order to manage update of polygon definition if the cell is executed more than once
    raster_patch.visible = False
except:
    pass
raster_patch = p.patches('x', 'y', source = ColumnDataSource(dict(x = geom_x, y = geom_y)), line_color = "orange", line_width = 0.5, fill_color="orange", fill_alpha=0.2, name="raster_patch")

show(p)